# 1. DATA GENERATION

In [ ]:
import sys
sys.path.append("../raw_audio/end2you")

from end2you.generation_process import GenerationProcess
from end2you.utils import Params

## Training data

In [ ]:
generator_params_train = Params(dict_params={
            'save_data_folder': '../raw_audio/5sec/hdf5/train',
            'num_outputs': 11,
            'modality': 'audio',
            'input_file': '../datapaths/trainpaths.csv',
            'exclude_cols': '0',
            'fieldnames': None,
            'delimiter': ',',
            'log_file': 'generation.log',
            'root_dir': '../raw_audio/5sec_final/outputs',
            'cuda': True
        })

In [ ]:
generation = GenerationProcess(generator_params_train)
generation.start()

In [ ]:
generator_params_devel = Params(dict_params={
            'save_data_folder': '../raw_audio/5sec/hdf5/devel',
            'num_outputs': 11,
            'modality': 'audio',
            'input_file': '../datapaths/develpaths.csv',
            'exclude_cols': '0',
            'fieldnames': None,
            'delimiter': ',',
            'log_file': 'generation_devel.log',
            'root_dir': '../raw_audio/5sec/outputs',
            'cuda': True
        })

In [ ]:
generation_devel = GenerationProcess(generator_params_devel)
generation_devel.start()

In [ ]:
generator_params_test = Params(dict_params={
            'save_data_folder': '../raw_audio/5sec/hdf5/test',
            'num_outputs': 11,
            'modality': 'audio',
            'input_file': '../datapaths/testpaths.csv',
            'exclude_cols': '0',
            'fieldnames': None,
            'delimiter': ',',
            'log_file': 'generation_test.log',
            'root_dir': '../raw_audio/5sec/outputs',
            'cuda': True
        })

In [ ]:
generation_test = GenerationProcess(generator_params_test)
generation_test.start()

# 3. Training

In [ ]:
import numpy as np

import sys
sys.path.append("../raw_audio/end2you")

from pathlib import Path
from end2you.training_process import TrainingProcess
from end2you.utils import Params

params = Params(dict_params={
    
    'train':Params(dict_params={'loss':'ce',
                                'dataset_path':'../raw_audio/5sec/hdf5/train',
                                'optimizer':'adagrad',
                                'learning_rate':0.002,
                                'summarywriter_file':'train_sw',
                                'num_epochs':50,
                                'num_workers':1,
                                'cuda':True, 
                                'modality':'audio',
                                'batch_size':10,
                                'is_training':True, # Always True
                                'save_summary_steps':10, 
                                'seq_length': 10
                               }),
    'valid':Params(dict_params={'summarywriter_file':'eval_sw',
                                'metric':'mse',
                                'dataset_path':'../raw_audio/5sec/hdf5/minidevel',
                                'num_workers':1,
                                'cuda':True,  
                                'modality':'audio',
                                'batch_size':1, # Always 1
                                'save_summary_steps':1, # Always 1
                                'is_training':False, # Always False
                                'seq_length': 10
                              }),
    'model':Params(dict_params={'model_name':'emo18', 'num_outs':11}),
    'root_dir':'../raw_audio/5sec/outputs3',
    'log_file':'training.log',
    'ckpt_path': None,
    'num_gpus':1
})
training = TrainingProcess(params)
training.start()

# 4. Evaluation

In [ ]:
import sys
sys.path.append("../raw_audio/end2you")

from end2you.generation_process import GenerationProcess
from end2you.utils import Params

import numpy as np

from pathlib import Path
from end2you.evaluation_process import EvaluationProcess
from end2you.utils import Params

params = Params(dict_params={
    'metric': 'mse',
    'modality': 'audio',
    'dataset_path':'../raw_audio/5sec/hdf5/test',
    'model_path':'../raw_audio/5sec/outputs/model/best.pth.tar',
    'cuda':True,
    'num_gpus':1,
    'root_dir':'../raw_audio/5sec/outputs',
    'seq_length': 10,
    'num_workers':2,
    'log_file': 'evaluation.log',
    'model':Params(dict_params={'model_name':'emo18', 'num_outs':11})
})
eval_process = EvaluationProcess(params)
scores, predictions = eval_process.start()